# Resume NER
## Extract Information from Resumes using NER (Named Entity Recognition)

### Part 2 - NER with Spacy
In this second part of the challenge, we will be using the preprocessed data from part one to start training NER models. We will be using spacy (https://spacy.io/) here to "get our feet wet" with NER, as training spacy can be reasonably done on our laptops and does not yet necessarily require a GPU. Spacy is a powerful, effective, and resource-efficient NLP library - It might surprise us with its performance on the challenge!

We will run spacy's pretrained models on our data to get a feel for NER, and then we will perform some additional preprocessing on our data before we start training our own NER model using the labelled entities we have identified in part one. 
We will also explore evaluation metrics for NER, and decide how we want to quantify the performance of our trained models. 

* *If you need help setting up python or running this notebook, please get help from the  assistants to the professor*
* *It might be helpful to try your code out first in a python ide like pycharm before copying it an running it here in this notebook*
* *For solving the programming tasks, use the python reference linked here (Help->Python Reference) as well as Web-searches.* 

##### Reload preprocessed data
Here, we will load the data we saved in part one and save it to a variable. Provide code below to load the data and store it as a list in a variable. (Hint - use 'open' and the json module)

In [1]:
## import json module
import json
path = "./data/converted_resumes.json"
## TODO open file load as json and store in "resumes" variable
with open(path,encoding="utf8") as file:
    resumes = file.readlines()
    
all_resumes = []
for line in resumes:
    data = json.loads(line, encoding="utf8")
    all_resumes.append(data)

resumes = all_resumes
## TODO print length of loaded resumes list to be sure everything ok
print(len(resumes))

690


##### Take Spacy for a spin
Before we train our own NER model to recognize the resume-specific entities we want to capture, let's see how spacy's pretrained NER models do on our data. These pretrained models can't recognize our entities yet, but let's see how they do. Run the next code block to load spacy's English language model 


In [2]:
import spacy
nlp = spacy.load('en')
print(nlp)

Now we get the EntityRecognizer in the loaded nlp pipeline and display the labels it supports

In [3]:
ner = nlp.get_pipe('ner')
labels = ner.labels
print(labels)

('WORK_OF_ART', 'DATE', 'QUANTITY', 'EVENT', 'LAW', 'ORDINAL', 'NORP', 'CARDINAL', 'PERSON', 'TIME', 'FAC', 'PERCENT', 'LANGUAGE', 'PRODUCT', 'GPE', 'LOC', 'ORG', 'MONEY')


##### Question: What do the 'GPE', 'FAC' and 'NORP' labels stand for? (Tipp: use either the spacy.explain method, or google the spacy.io api docs) 
*Answer here*

GPE : for recognition of  Countries, cities, states

FAC : for recognition of  Buildings, airports, highways, bridges, etc.

NORP : for recognition of  Nationalities or religious or political groups

In [4]:
### TODO  if you choose to use spacy's 'explain' method to get the answer to the question above, provide your code here
explain_GPE = spacy.explain('GPE')
explain_FAC = spacy.explain('FAC')
explain_NORP = spacy.explain('NORP')
## print description of entities using spacy explain
print('for recognition of ', explain_GPE)
print('for recognition of ', explain_FAC)
print('for recognition of ', explain_NORP)

for recognition of  Countries, cities, states
for recognition of  Buildings, airports, highways, bridges, etc.
for recognition of  Nationalities or religious or political groups


As we can see, the entities are different than the entities we will train our custom model on. 
##### Question: what entities do you think this model will find in an example resume?
*Answer here* 

* LANGUAGE : when among the skills, there are also different languages
* ORG : the names of companies can be recognized 
* CARDINAL : the different numbers and digits can be identified
* PERSON : the names of the different candidates can be recognized by this label
* DATE : the number of years of experience can be seen as a "date" indicator, or at least some 4-digit numbers could be seen as dates


Now we will work with one of our resumes, and get spacy to tell us what entities it recognizes. Complete the code block below to get a single resume text out of our resume list. 

In [5]:
### TODO get a single resume text and print it out
restxt = resumes[0][0]
## print it out, removing extraneous spaces
print("\n".join(restxt.split('\n\n')))

Afreen Jamadar
Active member of IIIT Committee in Third year
Sangli, Maharashtra - Email me on Indeed: indeed.com/r/Afreen-Jamadar/8baf379b705e37c6
I wish to use my knowledge, skills and conceptual understanding to create excellent team
environments and work consistently achieving organization objectives believes in taking initiative
and work to excellence in my work.
WORK EXPERIENCE
Active member of IIIT Committee in Third year
Cisco Networking -  Kanpur, Uttar Pradesh
organized by Techkriti IIT Kanpur and Azure Skynet.
PERSONALLITY TRAITS:
• Quick learning ability
• hard working
EDUCATION
PG-DAC
CDAC ACTS
2017
Bachelor of Engg in Information Technology
Shivaji University Kolhapur -  Kolhapur, Maharashtra
2016
SKILLS
Database (Less than 1 year), HTML (Less than 1 year), Linux. (Less than 1 year), MICROSOFT
ACCESS (Less than 1 year), MICROSOFT WINDOWS (Less than 1 year)
ADDITIONAL INFORMATION
TECHNICAL SKILLS:
• Programming Languages: C, C++, Java, .net, php.
• Web Designing: HTML, XML

Extracting entities with spacy is easy with a pretrained model. We simply call the model (here 'nlp') with our text to get a spacy Document. See https://spacy.io/api/doc for more detail. 

Execute the code below to process the resume txt.

In [6]:
doc = nlp(restxt)

The doc object has a list of entities predicted by spacy 'ents'. We would like to loop through all of these entities and print their label and associated text to see what spacy predicted for this resume.

Complete the code below to do this. You will probably need to google the spacy api docs to find the solution (Tipp: look for 'Doc.ents'). Also, trying code in your ide (for example pycharm) before copying it here might help with exploring and debugging to find the solution. 

In [7]:
##TODO loop through the doc's entities, and print the label and text for each entity found. 
ents = doc.ents
for elem in ents:
    print('recognized label -->', elem.label_, ':',  elem.text)

recognized label --> GPE : Jamadar
recognized label --> ORG : IIIT Committee
recognized label --> DATE : Third year
recognized label --> ORG : Maharashtra
recognized label --> ORG : IIIT Committee
recognized label --> DATE : Third year
recognized label --> PERSON : Cisco Networking -  Kanpur
recognized label --> ORG : Uttar Pradesh
recognized label --> ORG : Techkriti IIT Kanpur
recognized label --> PERSON : Azure Skynet
recognized label --> PERSON : Quick
recognized label --> ORG : CDAC
recognized label --> DATE : 2017
recognized label --> GPE : Kolhapur
recognized label --> ORG : Maharashtra
recognized label --> DATE : 2016
recognized label --> ORG : HTML
recognized label --> DATE : Less than 1 year
recognized label --> PERSON : Linux
recognized label --> DATE : Less than 1 year
recognized label --> ORG : MICROSOFT
recognized label --> PERSON : ACCESS
recognized label --> PERSON : C++
recognized label --> PERSON : Java
recognized label --> ORG : XML
recognized label --> ORG : Operati

##### Questions: What is your first impression of spacy's NER based on the results above? Does it seem accurate/powerfull?
##### Does it make many mistakes? Do some entity types seem more accurate than others? 
*Answers here*

We can see that the recognized labels are clearly not the one we have. 
The accuracy is clearly very low.

Now as a comparison, we will list the entities contained in the resume's original annotated training data (remember, the existing annotations were created by a human-annotator, and not predicted by a machine like the entities predicted above) 

Complete the code below to do the following: 
* Access the 'entities' list of the example resume you chose, loop through the entities and print them out. 
* *Tip: one entity in the list is a tuple with the following structure: (12,1222,"label") where the first element is the start index of the entity in the resume text, the second element is the end index, and the third element is the label.
* Use this Tip to print out a formatted list of entities 



In [8]:
##TODO print original entities for one resume
res = resumes[1]
restext = res[0]
labeled_ents = res[1]["entities"]
## TODO print out formatted list of entity labels and text
for ent in labeled_ents:
    print("{:30} {}".format(ent[2], restext[ent[0]:ent[1]]) )

Skills                         ❖ Operating Environment: […] Windows95/98/XP/NT
❖ Database Tool: SQL Management Studio (MSSQL), Business
Development Studio, Visual studio 2005
❖ Database Language: SQL, PL/SQL
❖ Ticket Tracking Tool: Service Now
❖ Reporting Tools: MS Reporting Services, SAS
❖ Languages: C, C++, PL/SQL
Skills                         Database (3 years), SQL (3 years), Sql Dba
Graduation Year                2012
College Name                   Indira Gandhi Institute Of Technology
College Name                   B.Tech in Computer Science and Engineering in CSE
Companies worked at            Microsoft Corporation
Location                       Bengaluru
Companies worked at            HCL Technologies
Designation                    SQL DBA Analyst
Designation                    DBA Support Analyst
projects                        Finance Support
Companies worked at            Microsoft Corporation
Email Address                  indeed.com/r/Alok-Khandai/5be849e443b8f467
Compani

As we already know, the annotated entities in the training data are different than the entities spacy can recognize with it's pretrainied models, so we need to train a custom NER model. We will get started with that now. 

##### Prepare Training Data for NER model training
We need to do some more preprocessing of our training data before we can train our model.

Remember the entity labels you chose in part 1 of the challenge? We will be training a model to predict those entities.
As a first step, we will gather all resumes that contain at least one training annotation for those entities.

Complete and execute the code below to gather your training data. 

In [9]:
##TODO Store the entity labels you want to train for as array in chosen_entity_labels
chosen_entity_labels = ['Skills', 'Degree', 'Companies worked at']

## this method gathers all resumes which have all of the chosen entites above.
def gather_candidates(dataset,entity_labels):
    candidates = list()
    for resume in dataset:
        res_ent_labels = list(zip(*resume[1]["entities"]))[2]
        if set(entity_labels).issubset(res_ent_labels):
            candidates.append(resume)
    return candidates
## TODO use the gather candidates methods and store result in training_data variable
training_data = gather_candidates(resumes, chosen_entity_labels)
print("Gathered {} training examples".format(len(training_data)))

Gathered 437 training examples


Now we have those training examples which contain the entities we are interested in. Do you have at least a few hundred examples? If not, you might need to re-think the entities you chose or try just one or two of them and re-run the notebooks. It is important that we have several hundred examples for training (e.g. more than 200. 3-500 is better). 

##### Remove other entity annotations from training data
Now that we have our training data, we want to remove all but relevant (chosen) entity annotations from this data, so that the model we train will only train for our entities. Complete and execute the code below to do this. 

In [10]:
## filter all annotation based on filter list
def filter_ents(ents, filter):
    filtered = [ent for ent in ents if ent[2] in filter]
    return filtered

## TODO use method above to remove all but relevant (chosen) entity annotations and store in X variable 
ents = []
for data in training_data:
    all_labels = data[1]["entities"]
    for label in all_labels:
            ents.append(label)

# X = filter_ents(ents, chosen_entity_labels)
X = [[dat[0], dict(entities=filter_ents(dat[1]['entities'], chosen_entity_labels))] for dat in training_data]
print(len(X))
print(X[2][1])

437
{'entities': [[5088, 5109, 'Skills'], [5071, 5085, 'Skills'], [3781, 3795, 'Skills'], [3490, 3523, 'Skills'], [3388, 3420, 'Degree'], [3290, 3311, 'Degree'], [1742, 1749, 'Companies worked at'], [1491, 1497, 'Companies worked at'], [1289, 1296, 'Companies worked at'], [1257, 1264, 'Companies worked at'], [1223, 1230, 'Companies worked at'], [1192, 1199, 'Companies worked at'], [786, 818, 'Skills'], [768, 780, 'Skills'], [752, 763, 'Skills'], [675, 696, 'Skills'], [656, 670, 'Skills'], [475, 478, 'Degree'], [38, 58, 'Companies worked at'], [38, 44, 'Companies worked at']]}


##### Remove resumes that cause errors in spacy
Depending on what entities you chose, some of the resumes might cause errors in spacy. We don't need to get into details as to why, suffice to say it has to do with whitespace and syntax in the entity annotations. If these resumes are not removed from our training data, spacy will throw an exception during training, so we need to remove them first. 

We will use the remove_bad_data function below to do this. This function does the following:
* calls train_spacy_ner with debug=True and n_iter=1. This causes spacy to process the documents one-by-one, and gather the documents that throw an exception in a list of "bad docs" which it returns. 
* You will complete the function to remove any baddocs (returned by remove_bad_data) from your training data list. 

You may or may not have any bad documents depending on the entities you chose. In any case, there should not be more than a dozen or so bad docs.  

In [11]:
from spacy_train_resume_ner import train_spacy_ner

def remove_bad_data(training_data):
    model, baddocs = train_spacy_ner(training_data, debug=True, n_iter=1)
    ## training data is list of lists with each list containing a text and annotations
    ## baddocs is a set of strings/resume texts.
    ## TODO complete implementation to filter bad docs and store filter result (good docs) in filtered variable
    filtered = [data for data in training_data if data[0] not in baddocs]
    print("Unfiltered training data size: ",len(training_data))
    print("Filtered training data size: ", len(filtered))
    print("Bad data size: ", len(baddocs))
    return filtered

## call remove method. It may take a few minutes for the method to complete.
## you will know it is complete when the print output above.
X = remove_bad_data(X)

Created blank 'en' model
Losses {'ner': 43437.77209984537}
Unfiltered training data size:  437
Filtered training data size:  437
Bad data size:  0


##### Question: How many bad docs did you have? What is the size of your new (filtered) training data? 
*Answer here*

0 bad docs. the new size of our filtered training data is 437

##### Train/Test Split
Now before we train our model, we have to split our available training data into training and test sets. Splitting our data into train and test (or holdout) datasets is a fundamental technique in machine learning, and essential to avoid the problem of overfitting.
Before we go on, you should get a grasp of how train/test split helps us avoid overfitting. Please take the time now to do a quick web search on the topic. There are many resources available. You should search for "train test validation overfitting" or some subset of those terms.

Here are a few articles to start with:
* https://machinelearningmastery.com/a-simple-intuition-for-overfitting/
* https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation (Note: you are free to install scikit learn and use the train_test_split method documented here, but it is not necessary. It is the concept that is important)

##### Question: What is overfitting and how does doing a train/test split help us avoid overfitting when training our models? Please answer in your own words. 
*Answer here*

Overfitting refers to the fact that the predictive model produced by the Machine Learning algorithm adapts well to the Training Set.

train/test split allows us to see how our model behaves on data it has not yet seen. 
This notion is important because the purpose of Machine Learning is to predict results on data not seen by the predictive function. The goal of the game is that the prediction made is the closest possible to reality after the model has trained on learning data. 

Now that we understand why we do a train/test split, we will write some code that splits our data into train and test sets. Usually we want around 70-80% of the data for train, and the rest for test. 
##### TODO: Complete the code below to create a train and test dataset

In [12]:
##TODO complete the implementation  of the train test split function below
def train_test_split(X,train_percent):
    train_size = round(train_percent * len(X))
    train = X[0:train_size:]
    test = X[train_size::]
    return train,test
## TODO chose train size percent and call train test split, storing results in "train" and "test" variables.
train,test = train_test_split(X, 0.8)
## TODO use python assert to assert that the size of train and test sets add up to the size of all the data 
assert len(train) + len(test) == len(X)

##### Train a spacy ner model with our training data
OK, now it is (finally) time to train our own custom NER model using spacy. Because our training data has been preprocessed to only include annotations for the entities we are interested in, the model will only be able to predict/extract those entities. 
*Depending on your computer, this step may take a while.* Training 20 epochs (iterations) using 480 training examples takes around 10 minutes on my machine (core i7 CPU). You will see output like *Losses {'ner':2342.23342342}* after each epoch/iteration. The default number of iterations is 20, so you will see this output 20 times. When this step is done, we will use the trained ner model to perform predictions on our test data in our test set.  

In [13]:
## run this code to train a ner model using spacy
custom_nlp,_= train_spacy_ner(train,n_iter=20)

Created blank 'en' model
Losses {'ner': 37815.58809080891}
Losses {'ner': 35356.832652860816}
Losses {'ner': 23543.490550908842}
Losses {'ner': 39507.424731479085}
Losses {'ner': 34574.861733339596}
Losses {'ner': 37153.98180454783}
Losses {'ner': 37078.20448336378}
Losses {'ner': 30798.30722992555}
Losses {'ner': 21171.83980547829}
Losses {'ner': 24990.427372032776}
Losses {'ner': 25255.26998435636}
Losses {'ner': 24965.776010153582}
Losses {'ner': 20947.634760749876}
Losses {'ner': 18500.889791691676}
Losses {'ner': 16080.919177819625}
Losses {'ner': 15411.31817654447}
Losses {'ner': 15711.533377157111}
Losses {'ner': 12633.495664811351}
Losses {'ner': 12241.087240987443}
Losses {'ner': 11531.79968284876}


##### Inspect NER predictions on one sample resume
Now that we have a trained model, let's see how it works on one of our resumes. 

In [14]:
## TODO fetch one resume out of our test dataset and store to the "resume" variable
resume = test[0]
##   
restxt = resume[0]
doc = custom_nlp(restxt)
print(resume)

["Arunbalaji. R\nTechnical assistance and supervisor\n\nCHITLAPAKKAM, TAMIL NADU, IN - Email me on Indeed: indeed.com/r/Arunbalaji-\nR/9b55a0b150f23f61\n\nSeeking for a challenging position as a civil Engineer, where I can use my plannings, designing\nand overseeing skills in construction and help to grow the company to achieve its goal.\n\nWORK EXPERIENCE\n\nfreshers\n\nSpiro solution Pvt ltd -  Chennai, Tamil Nadu\n\nI'm working as a technical and designed engineer in Spiro solution pvt.ltd. I can manage a design\nand analytical problem in civil engineering,with dedication and effort.\n\nEDUCATION\n\nHSC in B.E.CIVIL ENGINEERING\n\nANNA UNIVERSITY\n\n2017\n\nSKILLS\n\nAUTO CAD (Less than 1 year), CADD (Less than 1 year), CONCRETE (Less than 1 year),\nCONFIDENT (Less than 1 year), DESIGN ENGINEERING (Less than 1 year)\n\nADDITIONAL INFORMATION\n\nKey skills:\n➢ Dedicated\n➢ Team player\n➢ Confident\n➢ Quick learner.\n\nSoftware proficiency:\n• Software known: Auto CADD, Staddpro, Revi

Now we will output the predicted entities and the existing annotated entities in that doc

In [15]:
## TODO output label and text of predicted entities (in "ents" variable of the spacy doc created above)
print("PREDICTED:")
ents = doc.ents
for ent in ents:
    print('recognized label -->', ent.label_, ':',  ent.text)
## TODO output labeled entities (in "entities" dictionary of resume)
print('\n##### ####\n')
print("LABELED:")
labeled_ents = resume[1]["entities"]
for ent in labeled_ents:
    print("{:30} {}".format(ent[2], restext[ent[0]:ent[1]]) )    

PREDICTED:
recognized label --> Companies worked at : Spiro solution Pvt ltd
recognized label --> Degree : HSC in B.E.CIVIL ENGINEERING
recognized label --> Skills : AUTO CAD
recognized label --> Skills : CADD
recognized label --> Skills : CONFIDENT
recognized label --> Skills : DESIGN ENGINEERING

##### ####

LABELED:
Skills                         L Ma
Skills                         t permissions like
Skills                         
❖ Exper
Skills                         cati
Skills                         
Databas
Degree                         ting DR solution, High Avail
Companies worked at            ronments.
❖ Working Ex


#### Evaluation Metrics for NER
Now that we can predict entities using our trained model, we can compare our predictions with the original annotations in our training data to evaluate how well our model performs for our task. The original annotations have been annotated manually by human annotators, and represent a "Gold Standard" against which we can compare our predictions. 

For most classification tasks, the most common evaluation metrics are:
* accuracy
* precision
* recall
* f1 score

In order to understand these metrics, we need to understand the following concepts:
* True positives - How many of the predicted entities are "true" according to the Gold Standard? (training annotation) 
* True negatives - How many entities did the model not predict which are actually not entities according to the Gold Standard?
* False positives - How many entities did the model predict which are NOT entities according to the Gold Standard?  
* False negatives - How many entities did the model "miss" - e.g. did not recognize as entities which are entities according to the Gold Standard? 

Before we go on, it is important that you understand true/false positives/negatives as well as the evaluation metrics above. Take some time now to research the web in order to find answers to the following questions:

##### Question: How are the evaluation metrics above defined in the context of evaluating Machine Learning models? How do they relate to True/False Positives/Negatives above? Please provide an intuitive description as well as the mathmatical formula for each metric. 
*Answers here*

**Accuracy** : calculates the percentage by which the classifier performs a correct classification (number of
correct classification / total number of prdiction made)
Accuracy = (TP + TN) / (TP + TN + FP + FN)

**Precision** : number of correct positive results / number of positive results predicted by the classifier.
Precision = TP / (TP + FP)

**Recall (sensitivity)** : number of correct positive results / number of all relevant samples
Recall = TP / (TP + FN)

**F1 Score** : weighted average of the precision and recall scores.
F1_score = (2 * precision * recall) / (precision + recall)

##### Calculating Metrics based on token-level annotations or full entity-level. 
The concepts above are our first step toward understanding how to evaluate our model effectively. However, in NER, we need to take into account that we can calculate our metrics either based on all tokens (words) found in the document, or only on the entities found in the document.  

##### Token-Level evaluation. 
Token level evaluation evaluates how accurately did the model tag *each individual word/token* in the input. In order to understand this, we need to understand something called the "BILUO" Scheme (or BILOU or BIO). The spacy docs have a good reference. Please read and familiarize yourself with BILUO. 

https://spacy.io/api/annotation#biluo

Up to now, we have not been working with the BILUO scheme, but with "offsets" (for example: (112,150,"Email") - which says there is an "Email" entity between positions 112 and 150 in the text). We would like to be able to evaluate our models on a token-level using BILUO - so we need to convert our data to BILUO. Fortunately, Spacy provides a helper method to do this for us.

*Execute the code below to see how our "Gold Standard" and predictions for our example doc above look in BILUO scheme.* 
Note: some of the lines might be ommited for display purposes. 

In [16]:
from spacy.gold import biluo_tags_from_offsets
import pandas as pd
from IPython.display import display, HTML

## returns a pandas dataframe with tokens, prediction, and true (Gold Standard) annotations of tokens
def make_bilou_df(nlp,resume):
    """
    param nlp - a trained spacy model
    param resume - a resume from our train or test set
    """
    doc = nlp(resume[0])
    bilou_ents_predicted = biluo_tags_from_offsets(doc, [(ent.start_char,ent.end_char,ent.label_)for ent in doc.ents])
    bilou_ents_true = biluo_tags_from_offsets(doc,[(ent[0], ent[1], ent[2]) for ent in resume[1]["entities"]])

    
    doc_tokens = [tok.text for tok in doc]
    bilou_df = pd.DataFrame()
    bilou_df["Tokens"] =doc_tokens
    bilou_df["Tokens"] = bilou_df["Tokens"].str.replace("\\s+","") 
    bilou_df["Predicted"] = bilou_ents_predicted
    bilou_df["True"] = bilou_ents_true
    return bilou_df

## TODO call method above with a resume from test set and store result in bilou_df variable.
bilou_df = make_bilou_df(custom_nlp, test[10])
display(bilou_df.head())  


,Tokens,Predicted,True
0,Ram,O,O
1,Edupuganti,O,O
2,,O,O
3,Software,O,O
4,Development,O,O


Based on this output, it should be very easy to calculate a token-level accuracy. We simply compare the "Predicted" to "True" columns and calculate what percentage are the same. 

In [17]:
## TODO bilou_df is a pandas dataframe. Use pandas dataframe api to get a subset where predicted and true are the same. 
same_df = bilou_df[bilou_df["Predicted"] == bilou_df["True"]]
## accuracy is the length of this subset divided by the length of bilou_df
accuracy = float(len(same_df) / len(bilou_df))
print("Accuracy on one resume: ",accuracy)


Accuracy on one resume:  0.9328512396694215


The accuracy might seem pretty good... if it is not 100%, then let's print out those tokens where the model predicted something different than the gold standard by running the code below. 

Note - if your score on one doc is 100%, pick another document and re-run the last few cells above. 

In [18]:
## TODO find all rows in bilou_df where "Predicted" not equal to "True" column. 
diff_df = bilou_df[bilou_df["Predicted"] != bilou_df["True"]]
display(diff_df)

,Tokens,Predicted,True
41,Oracle,U-Companies worked at,O
277,Oracle,O,-
278,Inc.,O,-
707,Relational,O,B-Skills
708,data,O,I-Skills
709,modeling,O,I-Skills
710,",",O,I-Skills
711,Object,O,I-Skills
712,Oriented,O,I-Skills
713,Modeling,O,I-Skills


Now let's calculate the accuracy on all our test resumes and average them for an accuracy score. 

Please complete the code below to report an accuracy score on our test resumes

In [19]:
import numpy as np
doc_accuracy = []
for res in test:
    ## TODO calculate accurac for each 'res' and append to doc_accuracy list 
    res_df = make_bilou_df(custom_nlp, res)
    res_same_df = res_df[res_df["Predicted"] == res_df["True"]]
    accuracy = float(len(res_same_df) / len(res_df))
    doc_accuracy.append(accuracy)

## TODO calculate mean/average of doc_accuracy (Tip: use numpy!)
total_acc = np.mean(doc_accuracy)
print("Accuracy: ",total_acc)


Accuracy:  0.8989647435628976


##### Question: how does the model perform on token-level accuracy? What did it miss? In those cases where the predictions didn't match the gold standard, were the predictions plausible or just "spurious" (wrong)? 
*Answer here* 

##### Question: What might the advantages and disadvantages be of calculating accuracy on token-level? Hint: think about a document with 1000 tokens where only 10 tokens are annotated as entities. What might the accuracy be on such a document?  

##### Entity-Level evaluation #####
Another method of evaluating the performance of our NER model is to calculate metrics not on token-level, but on entity level. There is a good blog article that describes this method. 

http://www.davidsbatista.net/blog/2018/05/09/Named_Entity_Evaluation/

The article goes into some detail, the most important part is the scenarios described in the section "Comparing NER system output and golden standard". 

##### Question: how do the first 3 scenarios described in the section "Comparing NER system output and golden standard" correlate to  true/false positives/negatives? 
*Answer here* 

##### Precision, Recall, F1 #####

Now we would like to calculate precision, recall, and f1 for each entity type we are interested in (our chosen entities). To do this, we need to understand the formulas for each. A good article for this is https://skymind.ai/wiki/accuracy-precision-recall-f1. 

##### Question: how can we calculate precision, recall and f1 score based on the information above? Please provide the formulas for each #####
*Answer here*

Precision : number of correct positive results / number of positive results predicted by the classifier. Precision = TP / (TP + FP)

Recall (sensitivity) : number of correct positive results / number of all relevant samples Recall = TP / (TP + FN)

F1 Score : weighted average of the precision and recall scores. F1_score = (2 * precision * recall) / (precision + recall)

Now supply code below which calculates precision and recall and F1 on our test data for each entity type we are interested in. 



In [20]:
## TODO cycle through chosen_entity_labels and calculate metrics for each entity using test data
data = []
for label in chosen_entity_labels:
    ## variables to store results for all resumes for one entity type
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    for tres in test:
        ## use make_bilou_df on each resume in our test set, and calculate for each entity true and false positives,
        ## and false negatives. 
        tres_df = make_bilou_df(custom_nlp, tres)
        ## calculate true false positives and false negatives for each resume
        tp = tres_df[(tres_df["Predicted"]==tres_df["True"]) & (tres_df["True"].str.contains(label))]
        fp = tres_df[(tres_df["Predicted"]!=tres_df["True"]) & (tres_df["Predicted"].str.contains(label))]
        fn = tres_df[(tres_df["Predicted"]!=tres_df["True"]) & (tres_df["True"].str.contains(label))]
        ## aggregate result for each resume to totals
        true_positives += len(tp)
        false_positives += len(fp)
        false_negatives += len(fn)
    
    print("For label '{}' tp: {} fp: {} fn: {}".format(label,true_positives,false_positives,false_negatives))
    
    ## TODO Use the formulas you learned to calculate metrics and print them out        
    precision = 0.0 if true_positives == 0 else true_positives / (true_positives + false_positives)
    recall = 0.0 if true_positives == 0 else true_positives / (true_positives + false_negatives)
    f1 = 0.0 if precision + recall == 0 else (2 * precision * recall) / (precision + recall)
    print("Precision: ",precision)
    print("Recall: ",recall)
    print("F1: ",f1)
    row = [precision,recall,f1]
    data.append(row)

## make pandas dataframe with metrics data. Use the chosen entity labels as an index, and the metric names as columns. 
metric_df = pd.DataFrame(data, index=chosen_entity_labels, columns=["Precision", "Recall", "f1"])
display(metric_df)

For label 'Skills' tp: 1133 fp: 1657 fn: 1515
Precision:  0.4060931899641577
Recall:  0.4278700906344411
F1:  0.4166973151894079
For label 'Degree' tp: 316 fp: 212 fn: 45
Precision:  0.5984848484848485
Recall:  0.8753462603878116
F1:  0.7109111361079865
For label 'Companies worked at' tp: 688 fp: 834 fn: 369
Precision:  0.4520367936925099
Recall:  0.6508987701040682
F1:  0.5335401318340443


,Precision,Recall,f1
Skills,0.406093,0.427870,0.416697
Degree,0.598485,0.875346,0.710911
Companies worked at,0.452037,0.650899,0.533540


Now we compute an average score for each metric. 

In [21]:
## TODO compute average metrics And print them out. Use pandas dataframe "mean" method to do this
print("Average precision:", metric_df["Precision"].mean())
print("Average recall:", metric_df["Recall"].mean())
print("Average f1:", metric_df["f1"].mean())

Average precision: 0.4855382773805053
Average recall: 0.6513717070421069
Average f1: 0.5537161943771463


##### Question: how do the average metrics here (computed on entity-level) compare to the token-level accuracy score above? Which metric(s) would you prefer to use to evaluate the quality of your model? Why? 

We are almost Done with part II! We just need to save our BILUO training data for reuse in Part III. For part III we are using flair and loading our data from a .csv file into a flair "Corpus". This is described here:

https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_6_CORPUS.md#reading-your-own-sequence-labeling-dataset


We need to create "train" and "test" .csv files using our train and test dataset which corresponds to the format described above. This format is one line containing (minimally) a text token and a NER Tag. These should be separated by whitespace.


"Sentences"
Flair works with "Sentences" which is a list of tokens. If we simply write out our csv with one line for every token in our dataset, we will have 1 giant sentence with many thousands of words.. This is not what we want. We would like to partition our data so that we have a list of "Sentences" - corresponding to our intuition for a sentence - a sequence of words that belong together and is not all to long, usually separated by some punctuation. When we create our .csv strings/files, we need to do so so that they represent a list of sentences, each sentence consisting of a list of tokens/tags (each token/tag being one line in our csv).


To do this, create a blank newline after each sentence.


How you split your resume data into sentences is your decision however.


Some starter code is provided below. Complete it to create two strings which you will save as .csv. Each file will be a (long) list of token/tag lines, with sentences separated by newlines.

In [22]:
def make_file(df):
    ## TODO - remove unwanted whitespace and/or newline token rows from dataframe
    regex = '^[ \n\r\s\d\W]*$'
    df = df.drop('Predicted', axis=1)
    df = df[~df['Tokens'].str.contains(regex)]
    df = pd.DataFrame({'sentences': df.iloc[:,:].apply(lambda x: ' '.join(x), axis=1)})
        
    ## TODO - insert newlines after each "sentence" 
        
    ## You can use df.to_csv below to create a csv for your data
    as_csv = df.to_csv(None, sep=" ", encoding="utf-8", index=False, header=False, line_terminator="\n\n")
        
    for c in [' B-', ' L-', ' I-', ' U-']:
        as_csv = as_csv.replace(c, ' ')
        
    return as_csv


def bilou_for_flair(nlp, train, test):
    """
    make .csv strings from train and test for use in flair
    """    
    print("Make bilou dfs")
    ## makes a list of pandas dataframes, one for each resume. 
    training_data_as_bilou = [make_bilou_df(nlp,res) for res in train]
    test_data_as_bilou = [make_bilou_df(nlp,res) for res in test]
    print("Done!")
    ## strings to return
    training_file = ""
    test_file = ""
    
    for idx,df in enumerate(training_data_as_bilou):
        ## training_file = TODO append csv string for each resume to big training file (dont forget newline after each)
        training_file += make_file(df) + "\n"
        
    for idx, df in enumerate(test_data_as_bilou):
        ## TODO same as above but for testfile
        test_file += make_file(df) + "\n"
        
    return training_file.replace('"', ''), test_file.replace('"', '')

training,test = bilou_for_flair(custom_nlp, train, test)
## TODO save training and test as .csv (wherever you want)
with open("./data/flair/train_res_bilou.txt", 'w', encoding="utf-8") as file:
        file.write(training)
        
with open("./data/flair/test_res_bilou.txt", 'w', encoding="utf-8") as file:
        file.write(test)

print('Save Done!!!')

Make bilou dfs
Done!
Save Done!!!


Now let's load the data we persisted with flair before we go on

In [23]:
# imports 
from flair.datasets import Corpus
from flair.data_fetcher import NLPTaskDataFetcher

# folder where training and test data are
data_folder = './data/flair/'

# your training file name
train_file = "train_res_bilou.txt"

# your training file name
test_file = "test_res_bilou.txt"

## TODO how is your file structured? {}
columns = {0: 'token', 1: 'label'}


## Now load our csv into flair corpus
corpus = NLPTaskDataFetcher.load_column_corpus(data_folder,column_format=columns,
                                               train_file=train_file,
                                               test_file=test_file)
print(corpus)

2019-06-18 18:21:48,857 Reading data from data/flair
2019-06-18 18:21:48,858 Train: data/flair/train_res_bilou.txt
2019-06-18 18:21:48,859 Dev: None
2019-06-18 18:21:48,859 Test: data/flair/test_res_bilou.txt


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated function (or staticmethod) load_column_corpus. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/flair/data_fetcher.py:312: DeprecationWarning: Call to deprecated function (or staticmethod) read_column_data. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  train_file, column_format
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/flair/data_fetcher.py:318: DeprecationWarning: Call to deprecated function (or staticmethod) read_column_data. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  test_file, column_format


Corpus: 167449 train + 18606 dev + 48803 test sentences


If you could load the corpus without error, you are ready to go on to part 3, where we will work with flair nlp! 